<h1>Outcomes Using Trust Features</h1>
Does performance improve for tasks (code status, leaving AMA, and in-hosp mortality) when adding mistrust features on top of demographics? 
Yes

In [1]:
import psycopg2
import pandas as pd
from time import gmtime, strftime
import tqdm

con = psycopg2.connect(dbname ='mimic', user='wboag', host="/var/run/postgresql")
cur = con.cursor()

In [2]:
print strftime("%Y-%m-%d %H:%M:%S", gmtime())

# LABEL: code status

code_query = "select distinct hadm_id,label,value from mimiciii.chartevents c JOIN mimiciii.d_items i on i.itemid=c.itemid where label = 'Code Status'"
code_status = pd.read_sql_query(code_query, con)

# binary labels
code_labels = {}
for i,row in tqdm.tqdm(code_status.iterrows()):
    if row.value is not None:
        if ('DNR' in row.value) or ('DNI' in row.value) or ('Comfort' in row.value) or ('Do Not' in row.value):
            label = 'DNR/CMO'
        elif (row.value == 'Full Code') or (row.value == 'Full code'):
            label = 'Full Code'
    code_labels[row.hadm_id] = label
    
#ode_status.head()

2019-01-05 04:15:57


46121it [00:06, 6879.72it/s]


In [3]:
print set(code_status['value'].values)

set(['DNR / DNI', 'DNR (do not resuscitate)', 'Other/Remarks', 'DNI (do not intubate)', 'Comfort measures only', 'Do Not Intubate', None, 'Full code', 'Full Code', 'Comfort Measures', 'CPR Not Indicate', 'Do Not Resuscita'])


In [4]:
# hadm -> race
import tqdm

def normalize_race(race):
    if 'HISPANIC' in race:
        return 'Hispanic'
    if 'SOUTH AMERICAN' in race:
        return 'Hispanic'
    if 'AMERICAN INDIAN' in race:
        return 'Native American'
    if 'ASIAN' in race:
        return 'Asian'
    if 'BLACK' in race:
        return 'Black'
    if 'WHITE' in race:
        return 'White'
    return 'Other'

def normalize_insurance(insurance):
    if insurance in ['Medicare', 'Medicaid', 'Government']:
        return 'Public'
    else:
        return insurance

In [5]:
# LABEL: left hospital against medical advice

# query for discharge info
discharge_query = 'SELECT distinct hadm_id,discharge_location FROM mimiciii.admissions'
discharge = pd.read_sql_query(discharge_query, con)

# binary labels
ama_labels = {}
for i,row in tqdm.tqdm(discharge.iterrows()):
    if row.discharge_location == 'LEFT AGAINST MEDICAL ADVI':
        label = 'AMA'
    else:
        label = 'compliant'
    ama_labels[row.hadm_id] = label

#discharge.head()

58976it [00:05, 10526.82it/s]


In [6]:
# LABEL: in-hospital mortality

# query for discharge info
mortality_query = 'SELECT distinct hadm_id,hospital_expire_flag FROM mimiciii.admissions'
mortality = pd.read_sql_query(mortality_query, con)

# binary labels
mortality_labels = {}
for i,row in tqdm.tqdm(mortality.iterrows()):
    if row.hospital_expire_flag:
        label = 'deceased'
    else:
        label = 'survived'
    mortality_labels[row.hadm_id] = label

#mortality.head()

58976it [00:04, 11829.58it/s]


In [7]:
import random

def data_split(ids, ratio=0.6):
    random.shuffle(ids)
    train = ids[:int(len(ids)*ratio) ]
    test  = ids[ int(len(ids)*ratio):]
    return train, test

In [8]:
# write informative features code

def analyze(task, vect, clf, count_top=False):

    ind2feat =  { i:f for f,i in vect.vocabulary_.items() }

    # create a 2-by-m matrix for biary, rather than relying on 1-p bullshit
    coef_ = clf.coef_
    
    # most informative features
    #"""
    print task
    informative_feats = np.argsort(coef_)
    
    if len(informative_feats.shape) == 2:
        informative_feats = informative_feats[0,:]
        coef_ = coef_[0,:]
        
    #'''
    # display what each feature is
    for feat in reversed(informative_feats):
        val = coef_[feat]

        word = ind2feat[feat]
        print '\t%-25s: %7.4f' % (word,val)
        

In [9]:
%matplotlib inline

import numpy as np
import sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import pylab as plt


def compute_stats(task, pred, P, ref, labels_map, verbose):
    if len(labels_map) == 2:
        scores = P[:,1] - P[:,0]
        res = compute_stats_binary(    task, pred, scores, ref, labels_map, verbose)
    else:
        res = compute_stats_multiclass(task, pred, P     , ref, labels_map, verbose)
    return res



def compute_stats_binary(task, pred, P, ref, labels, verbose):
    # santiy check
    assert all(map(int,P>0) == pred)

    V = [0,1]
    n = len(V)
    assert n==2, 'sorry, must be exactly two labels (how else would we do AUC?)'
    conf = np.zeros((n,n), dtype='int32')
    for p,r in zip(pred,ref):
        conf[p][r] += 1

    if verbose:
        print conf
        print
    
    tp = conf[1,1]
    tn = conf[0,0]
    fp = conf[1,0]
    fn = conf[0,1]

    precision   = tp / (tp + fp + 1e-9)
    recall      = tp / (tp + fn + 1e-9)
    sensitivity = tp / (tp + fn + 1e-9)
    specificity = tn / (tn + fp + 1e-9)

    f1 = (2*precision*recall) / (precision+recall+1e-9)

    tpr =  true_positive_rate(pred, ref)
    fpr = false_positive_rate(pred, ref)

    accuracy = (tp+tn) / (tp+tn+fp+fn + 1e-9)
    
    if verbose:
        print '\tspecificity %.3f' % specificity
        print '\tsensitivty: %.3f' % sensitivity

    # AUC
    if len(set(ref)) == 2:
        auc = sklearn.metrics.roc_auc_score(ref, P)
        if verbose: print '\t\tauc:        %.3f' % auc

    if verbose:
        print '\taccuracy:   %.3f' % accuracy
        print '\tprecision:  %.3f' % precision
        print '\trecall:     %.3f' % recall
        print '\tf1:         %.3f' % f1
        print '\tTPR:        %.3f' % tpr
        print '\tFPR:        %.3f' % fpr

        print 'TODO: VIZ THE ROC CURVE'

    res = {'accuracy':accuracy, 'precision':precision, 'recall':recall, 'f1':f1, 'tpr':tpr,
           'fpr':fpr, 'auc':auc, 'sensitivity':sensitivity, 'specificity':specificity}

    return res



def compute_stats_multiclass(task, pred, P, ref, labels_map):
    # santiy check
    assert all(map(int,P.argmax(axis=1)) == pred)

    # get rid of that final prediction dimension
    #pred = pred[1:]
    #ref  =  ref[1:]

    V = set(range(len(labels_map)))
    n = max(V)+1
    conf = np.zeros((n,n), dtype='int32')
    for p,r in zip(pred,ref):
        conf[p][r] += 1


    labels = [label for label,i in sorted(labels_map.items(), key=lambda t:t[1])]


    print conf
    print
    
    precisions = []
    recalls = []
    f1s = []
    print '\t prec  rec    f1   label'
    for i in range(n):
        label = labels[i]

        tp = conf[i,i]
        pred_pos = conf[i,:].sum()
        ref_pos  = conf[:,i].sum()

        precision   = tp / (pred_pos + 1e-9)
        recall      = tp / (ref_pos + 1e-9)
        f1 = (2*precision*recall) / (precision+recall+1e-9)

        print '\t%.3f %.3f %.3f %s' % (precision,recall,f1,label)

        # Save info
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)

    avg_precision = sum(precisions) / len(precisions)
    avg_recall    = sum(recalls   ) / len(recalls   )
    avg_f1        = sum(f1s       ) / len(f1s       )
    print '\t--------------------------'
    print '\t%.3f %.3f %.3f avg' % (avg_precision,avg_recall,avg_f1)

    print 'TODO: VIZ THE F1S'

    
    res = {'precisions':precisions, 'recalls':recalls, 'f1s':f1s}

    return res



def true_positive_rate(pred, ref):
    tp,fn = 0,0
    for p,r in zip(pred,ref):
        if p==1 and r==1:
            tp += 1
        elif p==0 and r==1:
            fn += 1
    return tp / (tp + fn + 1e-9)


def false_positive_rate(pred, ref):
    fp,tn = 0,0
    for p,r in zip(pred,ref):
        if p==1 and r==0:
            fp += 1
        elif p==0 and r==0:
            tn += 1
    return fp / (fp + tn + 1e-9)




def classification_results(svm, labels_map, X, Y, task, verbose=True):

    # for AUC
    P_ = svm.decision_function(X)

    # sklearn has stupid-ass changes in API when doing binary classification. make it conform to 3+
    if len(labels_map)==2:
        m = X.shape[0]
        P = np.zeros((m,2))
        P[:,0] = -P_
        P[:,1] =  P_
    else:
        P = P_

    train_pred = P.argmax(axis=1)

    # what is the predicted vocab without the dummy label?
    V = labels_map.keys()

    if verbose: print task
    res = compute_stats(task, train_pred, P, Y, labels_map, verbose)
    if verbose: print '\n'
    return res
    


def regression_results(lr, test_X, test_Y, description, verbose=True):
    res = {}
    
    pred_Y = lr.predict(test_X)
    res['rms'] = sqrt(mean_squared_error(test_Y, pred_Y))
    res['mas'] = mean_absolute_error(test_Y, pred_Y)
    if verbose:
        print description
        print '\tRMS:', res['rms']
        print '\tMAS:', res['mas']
        print
    
        fig = plt.figure()
        perfect = np.arange(min(test_Y),max(test_Y),100)
        plt.scatter(perfect, perfect, color='red', s=0.01)
        plt.scatter(test_Y , pred_Y, color='blue', s=1)
        plt.xlabel('actual')
        plt.ylabel('prediction')
        plt.show()
    
    return res

In [15]:
# Load features

import cPickle as pickle

def normalize(scores):
    vals = np.array(scores.values())
    mu = vals.mean()
    std = vals.std()
    return { k:(v-mu)/std for k,v in scores.items()}


# query for insurance info
insurance_query = 'SELECT distinct hadm_id,insurance FROM mimiciii.admissions'
insurance = pd.read_sql_query(insurance_query, con)

# query for oasis info
oasis_query = 'SELECT distinct hadm_id,oasis FROM mimiciii.oasis'
oasis = pd.read_sql_query(oasis_query, con)

# query for demographics info
patients_query = 'SELECT distinct hadm_id,gender,age,ethnicity,admission_type,los_hospital FROM mimiciii.icustay_detail'
patients = pd.read_sql_query(patients_query, con)
patients = patients.loc[patients['admission_type']!='NEWBORN']

# Load trust scores
with open('../data/mistrust_noncompliant.pkl', 'rb') as f:
    noncompliant_dict = normalize(pickle.load(f))
print 'noncompliant:', len(noncompliant_dict)
noncompliant_df = pd.DataFrame(noncompliant_dict.items(), columns=['hadm_id','noncompliant'])

# Load trust scores
with open('../data/mistrust_autopsy.pkl', 'rb') as f:
    autopsy_dict = normalize(pickle.load(f))
print 'autopsy:', len(autopsy_dict)
autopsy_df = pd.DataFrame(autopsy_dict.items(), columns=['hadm_id','autopsy'])

# Load trust scores
with open('../data/neg_sentiment.pkl', 'rb') as f:
    sentiment_dict = normalize(pickle.load(f))
print 'sentiment:', len(sentiment_dict)
sentiment_df = pd.DataFrame(sentiment_dict.items(), columns=['hadm_id','sentiment'])

    
# merge data
extra_1 = pd.merge(insurance, oasis, on=['hadm_id'])
extra_2 = pd.merge(extra_1, noncompliant_df, on=['hadm_id'])
extra_3 = pd.merge(extra_2, autopsy_df     , on=['hadm_id'])
extra_4 = pd.merge(extra_3, sentiment_df   , on=['hadm_id'])
demographics = pd.merge(extra_4, patients  , on=['hadm_id'])

# Normalize some columns
demographics['ethnicity'] = demographics['ethnicity'].apply(normalize_race)
demographics['insurance'] = demographics['insurance'].apply(normalize_insurance)
demographics = demographics.rename(columns={'ethnicity':'race'})
demographics = demographics.rename(columns={'los_hospital':'los'})

#demographics.head()

noncompliant: 54510
autopsy: 54510
sentiment: 52726


In [16]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer

print strftime("%Y-%m-%d %H:%M:%S")


def normalize_mean_std(value, mu, std):
    return (value-mu)/std
    
# normalize ages
ages = np.array(demographics['age'])
age_mu = ages.mean()
age_std = ages.std()
demographics['age'] = demographics['age'].apply(lambda val:normalize_mean_std(val,age_mu,age_std))

# normalize oasis scores
oasis = np.array(demographics['oasis'])
oasis_mu = oasis.mean()
oasis_std = oasis.std()
demographics['oasis'] = demographics['oasis'].apply(lambda val:normalize_mean_std(val,oasis_mu,oasis_std))

# normalize los scores
los = np.array(demographics['los'])
los_mu = los.mean()
los_std = los.std()
demographics['los'] = demographics['los'].apply(lambda val:normalize_mean_std(val,los_mu,los_std))

# foo

def build_features(enabled):
    demographics_features = {}
    for i,row in tqdm.tqdm(demographics.iterrows()):
        feats = {}

        if 'admission_type' in enabled: feats[('admission_type', row.admission_type   )] = 1
        if 'oasis'          in enabled: feats[('oasis', None)] = row.oasis

        if 'age' in enabled: feats[('age'  , None)] = row.age
        if 'los' in enabled: feats[('los'  , None)] = row.los

        if 'insurance' in enabled: feats[('insurance'     , row.insurance)] = 1
        if 'gender'    in enabled: feats[('gender'        , row.gender   )] = 1

        if 'race'     in enabled: feats[('race', row.race     )] = 1
            
        if 'noncompliant' in enabled: feats[('concompliant',None)] = row.noncompliant
        if 'autopsy'      in enabled: feats[('autopsy'     ,None)] = row.autopsy
        if 'sentiment'    in enabled: feats[('sentiment'   ,None)] = row.sentiment

        demographics_features[row.hadm_id] = feats

    print strftime("%Y-%m-%d %H:%M:%S")

    # fit vectorizer
    vect = DictVectorizer()
    vect.fit(demographics_features.values())
    print 'num_features:', len(vect.get_feature_names())

    # ordering of all features
    ids = demographics_features.keys()
    print '\t', strftime("%Y-%m-%d %H:%M:%S")
    X = vect.transform([demographics_features[hadm_id] for hadm_id in ids])    

    return demographics_features, vect
    
print strftime("%Y-%m-%d %H:%M:%S")

2019-01-04 23:55:40
2019-01-04 23:55:40


In [17]:
# AMA
from collections import defaultdict, Counter

print strftime("%Y-%m-%d %H:%M:%S")
from sklearn.linear_model import LogisticRegression



featlists = {
                #'BASELINE'             :['age', 'los', 'insurance', 'gender'],
                #'BASELINE+RACE'        :['age', 'los', 'insurance', 'gender', 'race'],
                #'BASELINE+NONCOMPLIANT':['age', 'los', 'insurance', 'gender', 'noncompliant'],
                #'BASELINE+AUTOPSY'     :['age', 'los', 'insurance', 'gender', 'autopsy'],
                #'BASELINE+SENTIMENT'   :['age', 'los', 'insurance', 'gender', 'sentiment'],
                'BASELINE+ALL'         :['age', 'los', 'insurance', 'gender', 'race', 'noncompliant', 'autopsy', 'sentiment']
            }

ama_Y_vect = {'AMA': 1, 'compliant': 0}

feature_weights = defaultdict(list)

for name,featlist in featlists.items():
    print name
    print featlist
    
    demographics_features, vect = build_features(featlist)
    ind2feat =  { i:f for f,i in vect.vocabulary_.items() }

    ama_ids = list(set(discharge['hadm_id'].values) & set(demographics_features.keys()))
    print 'patients:', len(ama_ids)
  
    print Counter([ama_Y_vect[ama_labels[hadm_id]] for hadm_id in ama_ids])

    aucs = []
    for iteration in tqdm.tqdm(range(100)):

        # train/test split
        ama_train_ids, ama_test_ids = data_split(ama_ids)

        # select pre-computed features
        ama_train_features = [demographics_features[hadm_id] for hadm_id in ama_train_ids]
        ama_test_features  = [demographics_features[hadm_id] for hadm_id in ama_test_ids ]

        # vectorize features
        ama_train_X = vect.transform(ama_train_features)
        ama_test_X  = vect.transform(ama_test_features)

        # vectorize task-specific labels
        #print ama_Y_vect

        # select labels
        ama_train_Y = [ama_Y_vect[ama_labels[hadm_id]] for hadm_id in ama_train_ids]
        ama_test_Y  = [ama_Y_vect[ama_labels[hadm_id]] for hadm_id in ama_test_ids ]

        # fit model
        ama_svm = LogisticRegression(C=0.1, penalty='l1', tol=0.01)
        ama_svm.fit(ama_train_X,ama_train_Y)
        #print ama_svm


        # AMA Model eval

        # evaluate model
        res = classification_results(ama_svm, ama_Y_vect,  ama_test_X,  ama_test_Y, 'test:  ama', verbose=False)
        aucs.append(res['auc'])

        # record the weights of the features (because we average them)
        if name == 'BASELINE+ALL':
            for feat,val in enumerate(ama_svm.coef_.tolist()[0]):
                featname = ind2feat[feat]
                feature_weights[featname].append(val)

        #classification_results(ama_svm, ama_Y_vect, ama_train_X, ama_train_Y, 'train: ama')

    aucs = np.array(aucs)
    print 'AUCS: ', aucs
    print '    mean:     ', aucs.mean()
    print '    1.96*std: ', aucs.std() * 1.96
    print '    conf_interval: (%.4f,%.4f)' % (aucs.mean()-1.96*aucs.std(),aucs.mean()+1.96*aucs.std())


    # most informative features
    analyze('ama', vect, ama_svm)
    print '\n\n\n'

    if name == 'BASELINE+ALL':
        for featname,vals in sorted(feature_weights.items()):
            v = np.array(vals)
            mu = v.mean()
            std = v.std()
            print '%-10s:%-15s || %.2f +/- %.2f' % (featname[0],featname[1],mu,1.96*std)

print strftime("%Y-%m-%d %H:%M:%S")

156it [00:00, 1558.40it/s]

2019-01-04 23:55:43
BASELINE+ALL
['age', 'los', 'insurance', 'gender', 'race', 'noncompliant', 'autopsy', 'sentiment']


50976it [00:11, 4621.00it/s]


2019-01-04 23:55:54
num_features: 16
	2019-01-04 23:55:55


  0%|          | 0/100 [00:00<?, ?it/s]

patients: 47543
Counter({0: 47245, 1: 298})


/scratch/wboag/2019/cleanup/venv_2/local/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
100%|██████████| 100/100 [00:58<00:00,  1.41it/s]

AUCS:  [0.89085836 0.87451773 0.86329029 0.88408219 0.89035107 0.87634156
 0.85361896 0.88157711 0.88065734 0.86516597 0.86856116 0.88300858
 0.88137883 0.88320671 0.88812047 0.88007833 0.88230375 0.87829285
 0.8748245  0.91639247 0.89264881 0.89115691 0.87476056 0.85905941
 0.87554156 0.85732955 0.88284435 0.85839766 0.8799578  0.86908893
 0.89174152 0.86671688 0.88220822 0.87732697 0.88030455 0.89137241
 0.8867698  0.86312367 0.85721262 0.86575462 0.88775139 0.85696205
 0.87613734 0.87202482 0.8802392  0.90086206 0.85793807 0.86521937
 0.89495269 0.8658927  0.87831191 0.86941768 0.87711767 0.88482268
 0.87855656 0.88983952 0.89751318 0.8847497  0.85721101 0.88421151
 0.87548531 0.88819443 0.86884018 0.84322693 0.88361441 0.8869534
 0.87567408 0.85618495 0.88431146 0.88122912 0.86976073 0.89480167
 0.87060475 0.86481452 0.88576387 0.89274246 0.87193027 0.85798495
 0.87003016 0.86079918 0.87679576 0.87799497 0.89582342 0.86226321
 0.89043775 0.88136813 0.86984861 0.87225548 0.8670835  

In [18]:
# Code Status

print strftime("%Y-%m-%d %H:%M:%S")


from sklearn.linear_model import LogisticRegression



featlists = {
                #'BASELINE'             :['age', 'los', 'insurance', 'gender'],
                #'BASELINE+RACE'        :['age', 'los', 'insurance', 'gender', 'race'],
                #'BASELINE+NONCOMPLIANT':['age', 'los', 'insurance', 'gender', 'noncompliant'],
                #'BASELINE+AUTOPSY'     :['age', 'los', 'insurance', 'gender', 'autopsy'],
                #'BASELINE+SENTIMENT'   :['age', 'los', 'insurance', 'gender', 'sentiment'],
                'BASELINE+ALL'         :['age', 'los', 'insurance', 'gender', 'race', 'noncompliant', 'autopsy', 'sentiment']
            }

cs_Y_vect = {'DNR/CMO': 1, 'Full Code': 0}

feature_weights = defaultdict(list)

for name,featlist in featlists.items():
    print name
    print featlist
    
    demographics_features, vect = build_features(featlist)
    ind2feat =  { i:f for f,i in vect.vocabulary_.items() }
    
    print ind2feat

    cs_ids = list(set(code_labels.keys()) & set(demographics_features.keys()))
    print 'patients:', len(cs_ids)
    
    print Counter([cs_Y_vect[code_labels[hadm_id]] for hadm_id in cs_ids])

    
    aucs = []
    for iteration in tqdm.tqdm(range(100)):

        #print 'Iter:', iteration

        # train/test split
        cs_train_ids, cs_test_ids = data_split(cs_ids)

        # select pre-computed features
        cs_train_features = [demographics_features[hadm_id] for hadm_id in cs_train_ids]
        cs_test_features  = [demographics_features[hadm_id] for hadm_id in cs_test_ids ]

        # vectorize features
        cs_train_X = vect.transform(cs_train_features)
        cs_test_X  = vect.transform(cs_test_features)

        # vectorize task-specific labels
        cs_Y_vect = {'DNR/CMO': 1, 'Full Code': 0}
        #print cs_Y_vect

        # select labels
        cs_train_Y = [cs_Y_vect[code_labels[hadm_id]] for hadm_id in cs_train_ids]
        cs_test_Y  = [cs_Y_vect[code_labels[hadm_id]] for hadm_id in cs_test_ids ]

        # fit model
        cs_svm = LogisticRegression(C=0.1, penalty='l1', tol=0.01)
        cs_svm.fit(cs_train_X,cs_train_Y)
        #print cs_svm


        # cs Model eval

        # evaluate model
        res = classification_results(cs_svm, cs_Y_vect, cs_test_X,  cs_test_Y, 'test:  cs', verbose=False)
        aucs.append(res['auc'])

        # record the weights of the features (because we average them)
        if name == 'BASELINE+ALL':
            for feat,val in enumerate(cs_svm.coef_.tolist()[0]):
                featname = ind2feat[feat]
                feature_weights[featname].append(val)
            
        # most informative features
        #analyze('cs', vect, cs_svm)
        
    aucs = np.array(aucs)
    print 'AUCS: ', aucs
    print '    mean:     ', aucs.mean()
    print '    1.96*std: ', aucs.std() * 1.96
    print '    conf_interval: (%.4f,%.4f)' % (aucs.mean()-1.96*aucs.std(),aucs.mean()+1.96*aucs.std())


    # most informative features
    analyze('cs', vect, cs_svm)

    if name == 'BASELINE+ALL':
        for featname,vals in sorted(feature_weights.items()):
            v = np.array(vals)
            mu = v.mean()
            std = v.std()
            print '%-10s:%-15s || %.2f +/- %.2f' % (featname[0],featname[1],mu,1.96*std)
    
print strftime("%Y-%m-%d %H:%M:%S")

0it [00:00, ?it/s]

2019-01-04 23:56:54
BASELINE+ALL
['age', 'los', 'insurance', 'gender', 'race', 'noncompliant', 'autopsy', 'sentiment']


50976it [00:09, 5236.42it/s]


2019-01-04 23:57:03
num_features: 16
	2019-01-04 23:57:04


  0%|          | 0/100 [00:00<?, ?it/s]

{0: ('age', None), 1: ('autopsy', None), 2: ('concompliant', None), 3: ('gender', 'F'), 4: ('gender', 'M'), 5: ('insurance', 'Private'), 6: ('insurance', 'Public'), 7: ('insurance', 'Self Pay'), 8: ('los', None), 9: ('race', 'Asian'), 10: ('race', 'Black'), 11: ('race', 'Hispanic'), 12: ('race', 'Native American'), 13: ('race', 'Other'), 14: ('race', 'White'), 15: ('sentiment', None)}
patients: 39125
Counter({0: 36667, 1: 2458})


100%|██████████| 100/100 [00:56<00:00,  1.73it/s]

AUCS:  [0.7797595  0.7864562  0.79014992 0.77520486 0.78525847 0.77857338
 0.78509857 0.7884304  0.79088733 0.78313686 0.78970985 0.78382875
 0.78810281 0.78749812 0.78864029 0.79968577 0.79118141 0.7766818
 0.78442367 0.79835093 0.79320046 0.78782693 0.77922702 0.77785324
 0.77653062 0.7739758  0.77545866 0.78683858 0.78426325 0.79146758
 0.78851617 0.78034419 0.78179609 0.78053583 0.79187255 0.77957677
 0.77715471 0.78672934 0.78565199 0.78691719 0.7823756  0.79901502
 0.78399456 0.79059203 0.78916896 0.78086252 0.78317603 0.789163
 0.77354958 0.7843122  0.78671648 0.78893047 0.78814728 0.79108546
 0.78363495 0.78281243 0.78065414 0.78432409 0.78813066 0.79278352
 0.7875232  0.7914536  0.78602473 0.78989935 0.78689736 0.78563218
 0.77992618 0.78788532 0.7715448  0.77773195 0.78589674 0.7889953
 0.77955621 0.78865128 0.77682282 0.78573882 0.79123264 0.79180179
 0.79263443 0.78321795 0.77307465 0.78996017 0.7806325  0.78791611
 0.77808487 0.78548196 0.78705411 0.79317152 0.7917187  0.7

In [19]:
# Mortality

print strftime("%Y-%m-%d %H:%M:%S")


from sklearn.linear_model import LogisticRegression


featlists = {
                #'BASELINE'             :['age', 'los', 'insurance', 'gender'],
                #'BASELINE+RACE'        :['age', 'los', 'insurance', 'gender', 'race'],
                #'BASELINE+NONCOMPLIANT':['age', 'los', 'insurance', 'gender', 'noncompliant'],
                #'BASELINE+AUTOPSY'     :['age', 'los', 'insurance', 'gender', 'autopsy'],
                #'BASELINE+SENTIMENT'   :['age', 'los', 'insurance', 'gender', 'sentiment'],
                'BASELINE+ALL'         :['age', 'los', 'insurance', 'gender', 'race', 'noncompliant', 'autopsy', 'sentiment']
            }
mortality_Y_vect = {'deceased': 1, 'survived': 0}

feature_weights = defaultdict(list)


for name,featlist in featlists.items():
    print name
    print featlist
    
    demographics_features, vect = build_features(featlist)
    ind2feat =  { i:f for f,i in vect.vocabulary_.items() }

    mortality_ids = list(set(mortality_labels.keys()) & set(demographics_features.keys()))
    print 'patients:', len(mortality_ids)
    
    print Counter([mortality_Y_vect[mortality_labels[hadm_id]] for hadm_id in mortality_ids])

    aucs = []
    for iteration in tqdm.tqdm(range(100)):

        #print 'Iter:', iteration


        # train/test split
        mortality_train_ids, mortality_test_ids = data_split(mortality_ids)

        # select pre-computed features
        mortality_train_features = [demographics_features[hadm_id] for hadm_id in mortality_train_ids]
        mortality_test_features  = [demographics_features[hadm_id] for hadm_id in mortality_test_ids ]

        # vectorize features
        mortality_train_X = vect.transform(mortality_train_features)
        mortality_test_X  = vect.transform(mortality_test_features)

        # vectorize task-specific labels
        #print mortality_Y_vect

        # select labels
        mortality_train_Y = [mortality_Y_vect[mortality_labels[hadm_id]] for hadm_id in mortality_train_ids]
        mortality_test_Y  = [mortality_Y_vect[mortality_labels[hadm_id]] for hadm_id in mortality_test_ids ]

        # fit model
        mortality_svm = LogisticRegression(C=0.1, penalty='l1', tol=0.01)
        mortality_svm.fit(mortality_train_X,mortality_train_Y)
        #print mortality_svm


        # mortality Model eval

        # evaluate model
        res = classification_results(mortality_svm, mortality_Y_vect, mortality_test_X, mortality_test_Y, 'test:  mortality', verbose=False)
        aucs.append(res['auc'])

        # record the weights of the features (because we average them)
        if name == 'BASELINE+ALL':
            for feat,val in enumerate(mortality_svm.coef_.tolist()[0]):
                featname = ind2feat[feat]
                feature_weights[featname].append(val)
            
        # most informative features
        #analyze('mortality', vect, mortality_svm)
        
    aucs = np.array(aucs)
    print 'AUCS: ', aucs
    print '    mean:     ', aucs.mean()
    print '    1.96*std: ', aucs.std() * 1.96
    print '    conf_interval: (%.4f,%.4f)' % (aucs.mean()-1.96*aucs.std(),aucs.mean()+1.96*aucs.std())


    # most informative features
    analyze('mortality', vect, mortality_svm)

    # foo


    if name == 'BASELINE+ALL':
        for featname,vals in sorted(feature_weights.items()):
            v = np.array(vals)
            mu = v.mean()
            std = v.std()
            print '%-10s:%-15s || %.2f +/- %.2f' % (featname[0],featname[1],mu,1.96*std)
            
    print '\n\n'
    
print strftime("%Y-%m-%d %H:%M:%S")

0it [00:00, ?it/s]

2019-01-04 23:58:01
BASELINE+ALL
['age', 'los', 'insurance', 'gender', 'race', 'noncompliant', 'autopsy', 'sentiment']


50976it [00:09, 5101.28it/s]


2019-01-04 23:58:11
num_features: 16
	2019-01-04 23:58:11


  0%|          | 0/100 [00:00<?, ?it/s]

patients: 47543
Counter({0: 42429, 1: 5114})


100%|██████████| 100/100 [01:04<00:00,  1.54it/s]

AUCS:  [0.67597942 0.66796189 0.68214396 0.66944476 0.67395821 0.67354888
 0.67068775 0.67290468 0.66699559 0.66895833 0.67467822 0.67241983
 0.67598472 0.67080436 0.67627854 0.67294662 0.67546048 0.66784427
 0.66833621 0.67154392 0.67297642 0.68192794 0.67203645 0.67081948
 0.66367444 0.67145488 0.6727461  0.66890287 0.66642047 0.67044154
 0.66952413 0.66924098 0.67655431 0.67170851 0.66759812 0.67527588
 0.67156593 0.68010592 0.67829128 0.67254243 0.66715664 0.67592364
 0.6731543  0.6670147  0.66636753 0.6692419  0.68142159 0.67192896
 0.67877901 0.67566752 0.66671362 0.67740352 0.67710221 0.67190979
 0.67630895 0.67681584 0.67535052 0.67356308 0.67016684 0.66646245
 0.68222291 0.67226293 0.67685544 0.66950803 0.66685135 0.67073491
 0.67617529 0.66689762 0.66978465 0.66456813 0.67865127 0.67193433
 0.67314576 0.67493967 0.67532346 0.66796884 0.67105338 0.66678113
 0.66869323 0.66587796 0.66838179 0.67589294 0.66866678 0.66446579
 0.66724467 0.66733614 0.67077739 0.67085804 0.67256763

In [20]:

metrics = {'noncompliant':noncompliant_dict, 'autopsy':autopsy_dict, 'sentiment':sentiment_dict}

for metric,scores in metrics.items():
    print metric

    vals = sorted(scores.values())
    n = len(vals)
    t1 = vals[1*n/4]
    t2 = vals[2*n/4]
    t3 = vals[3*n/4]

    lowest  = [hadm_id for hadm_id,score in scores.items() if     score<=t1]
    highest = [hadm_id for hadm_id,score in scores.items() if t3< score    ]

    def mort_rate(label, hadm_ids):
        cohort = mortality.loc[mortality['hadm_id'].isin(hadm_ids)]
        print '\t', label, sum(cohort['hospital_expire_flag'].values)/float(len(cohort))

    mort_rate('most  trust', lowest)
    mort_rate('least trust', highest)

autopsy
	most  trust 0.13357775318206974
	least trust 0.09118384195783577
noncompliant
	most  trust 0.04738502163867087
	least trust 0.12881658123467796
sentiment
	most  trust 0.04483386436049158
	least trust 0.16614824368409073
